In [1]:
"""
simulation_chunk_0
"""

import os
import pandas as pd
from typing import Dict, List, Any

from hopp.utilities import ConfigManager
from hopp.utilities.keys import set_developer_nrel_gov_key
from hopp.tools.optimization import SystemOptimizer, LoadAnalyzer
from hopp.tools.analysis.bos import EconomicCalculator
from hopp.simulation.resource_files import ResourceDataManager

class HybridOptimizer:
    """Wrapper class for hybrid system optimization."""
    
    def __init__(self, 
                 yaml_file_path: str,
                 api_key: str,
                 email: str,
                 project_lifetime: int = 25,
                 discount_rate: float = 0.0588,
                 enable_flexible_load: bool = True,  
                 max_load_reduction_percentage: float = 0.2):
        """Initialize hybrid system optimizer."""
        self.yaml_file_path = yaml_file_path
        self.api_key = api_key
        self.email = email
        self.project_lifetime = project_lifetime
        self.discount_rate = discount_rate
        
        # Set up components
        self.resource_manager = ResourceDataManager(api_key, email)
        self.economic_calculator = EconomicCalculator(discount_rate, project_lifetime)
        self.system_optimizer = SystemOptimizer(
            yaml_file_path, 
            self.economic_calculator,
            enable_flexible_load=enable_flexible_load,
            max_load_reduction_percentage=max_load_reduction_percentage
        )
        
        # Default optimization bounds
        self.bounds = [
            (5000, 50000),    # PV capacity (kW)
            (1, 50),          # Number of wind turbines
            (5000, 30000),    # Battery energy capacity (kWh)
            (1000, 10000),    # Battery power capacity (kW)
            (17000, 30000)    # Grid interconnect capacity (kW)
        ]

    def process_location(self, 
                        latitude: float, 
                        longitude: float, 
                        location_id: str = "") -> Dict[str, Any]:
        """
        Process a single location for optimization.
        
        Args:
            latitude: Site latitude
            longitude: Site longitude
            location_id: Optional identifier for the location
            
        Returns:
            dict: Optimization results for the location
        """
        print(f"\nProcessing location {location_id} at ({latitude}, {longitude})")
        
        try:
            # Download resource data
            solar_path = self.resource_manager.download_solar_data(
                latitude, longitude, "2020"
            )
            wind_path = self.resource_manager.download_wind_data(
                latitude, longitude, "20200101", "20201231"
            )
            
            # Update YAML configuration
            config = self.system_optimizer.config_manager.load_yaml_safely(self.yaml_file_path)
            config['site']['data']['lat'] = latitude
            config['site']['data']['lon'] = longitude
            config['site']['solar_resource_file'] = solar_path.replace('\\', '/')
            config['site']['wind_resource_file'] = wind_path.replace('\\', '/')
            self.system_optimizer.config_manager.save_yaml_safely(config, self.yaml_file_path)
            
            # Define initial conditions (10% of range)
            initial_conditions = [
                [bound[0] + (bound[1] - bound[0]) * 0.1 for bound in self.bounds]
            ]
            
            # Run optimization
            best_result = self.system_optimizer.optimize_system(self.bounds, initial_conditions)
            
            if best_result:
                print("\nBest configuration found:")
                for key, value in best_result.items():
                    if key != "Penalized LCOE ($/kWh)":
                        print(f"{key}: {value:.3f}")
                
                return {
                    'Latitude': latitude,
                    'Longitude': longitude,
                    'Location ID': location_id,
                    **{k: v for k, v in best_result.items() if k != "Penalized LCOE ($/kWh)"}
                }
            else:
                print("Optimization failed to converge")
                return {}
                
        except Exception as e:
            print(f"Error processing location: {str(e)}")
            return {}
    
    def optimize_multiple_locations(self, 
                                  locations_df: pd.DataFrame,
                                  output_path: str = None) -> pd.DataFrame:
        """
        Optimize system configurations for multiple locations.
        
        Args:
            locations_df: DataFrame containing columns 'DEPOSIT_LATITUDE', 'DEPOSIT_LONGITUDE', 'DEPOSIT_UID'
            output_path: Optional path to save results CSV
            
        Returns:
            pd.DataFrame: Optimization results for all locations
        """
        results = []
        
        for _, row in locations_df.iterrows():
            result = self.process_location(
                latitude=row['DEPOSIT_LATITUDE'],
                longitude=row['DEPOSIT_LONGITUDE'],
                location_id=row['DEPOSIT_UID']
            )
            if result:
                results.append(result)
        
        if results:
            results_df = pd.DataFrame(results)
            if output_path:
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                results_df.to_csv(output_path, index=False)
                print(f"\nResults saved to {output_path}")
            return results_df
        else:
            print("No successful optimizations to report")
            return pd.DataFrame()

def main():
    """Example usage of HybridOptimizer."""
    # Configuration
    yaml_file_path = "../input_yaml/input_file_chunk_0.yaml"
    csv_path = "../deposit_data/auCopper_chunk_0.csv"
    output_path = "../simulation_results_modularised/combined_results_without_DR/lowest_cost_configs_chunk_0_PDR.csv"
    api_key = "ZaurwKOnwDUp8rMyNBIxI4XiBo3b7L5oruTi0VX3"
    email = "hanrong.h99@gmail.com"
    
    # Set API key
    set_developer_nrel_gov_key(api_key)
    
    # Initialize optimizer with flexible load settings
    optimizer = HybridOptimizer(
        yaml_file_path=yaml_file_path,
        api_key=api_key,
        email=email,
        enable_flexible_load=True,  # or False if you want to disable flexible load
        max_load_reduction_percentage=0.20  # Default 20%, adjust as needed
    )
    
    # Load location data
    locations = pd.read_csv(csv_path)
    
    # Run optimization
    results = optimizer.optimize_multiple_locations(
        locations_df=locations,
        output_path=output_path
    )
    
    if not results.empty:
        print("\nOptimization complete. Summary of results:")
        print(f"Total locations processed: {len(results)}")
        print(f"Average LCOE: {results['System LCOE ($/kWh)'].mean():.4f}")
        print(f"Best LCOE: {results['System LCOE ($/kWh)'].min():.4f}")

if __name__ == "__main__":
    main()

/home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/examples/parallel_simulations/scripts_modularised/log/hybrid_systems_2024-11-08T23.22.46.623487.log



Processing location AUS.NSW.333484 at (-33.5265, 149.1588)
Using existing solar data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar/-33.5265_149.1588_psmv3_60_2020.csv
Using existing wind data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/wind/-33.5265_149.1588_NASA_2020_60min_50m.srw



Best configuration found:
PV Capacity (kW): 29292.000
Wind Turbine Capacity (kW): 1000.000
Genset Capacity (kW): 17000.000
Battery Energy Capacity (kWh): 5000.000
Battery Power Capacity (kW): 2610.000
Total System Generation (kWh): 2554244484.780
Total PV Generation (kWh): 1343766814.903
Total Wind Generation (kWh): 9670029.576
Total Genset Generation (kWh): 1200807640.300
Total Battery Generation (kWh): -61724.192
Total CO2 emissions (tonne): 785928.601
System NPC ($): 142823213.986
System LCOE ($/kWh): 0.303
Deficit Hours Fixed: 7625.000
Total Load Reduction (kWh): 0.000
Total Load Served (kWh): 1969558247.650
Total Charging Prevented (kWh): 15706572.412
Demand Not Served (kWh): 0.000
Load Reduction Percentage: 0.000
Demand Met Percentage: 100.000
Project Lifetime (years): 25.000

Processing location AUS.NSW.333485 at (-31.1629, 145.6538)
Using existing solar data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar/-31.1629


Best configuration found:
PV Capacity (kW): 23616.000
Wind Turbine Capacity (kW): 9000.000
Genset Capacity (kW): 17000.000
Battery Energy Capacity (kWh): 5000.000
Battery Power Capacity (kW): 2373.000
Total System Generation (kWh): 2636724727.619
Total PV Generation (kWh): 1239359997.144
Total Wind Generation (kWh): 495333792.198
Total Genset Generation (kWh): 902030938.276
Total Battery Generation (kWh): -56863.015
Total CO2 emissions (tonne): 590379.249
System NPC ($): 121744047.945
System LCOE ($/kWh): 0.258
Deficit Hours Fixed: 7250.000
Total Load Reduction (kWh): 0.000
Total Load Served (kWh): 1969558247.650
Total Charging Prevented (kWh): 14158275.671
Demand Not Served (kWh): 0.000
Load Reduction Percentage: 0.000
Demand Met Percentage: 100.000
Project Lifetime (years): 25.000

Processing location AUS.NSW.334161 at (-31.5001, 145.8422)
Using existing solar data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar/-31.500


Best configuration found:
PV Capacity (kW): 22846.000
Wind Turbine Capacity (kW): 9000.000
Genset Capacity (kW): 17000.000
Battery Energy Capacity (kWh): 5000.000
Battery Power Capacity (kW): 2356.000
Total System Generation (kWh): 2592088308.780
Total PV Generation (kWh): 1190684776.579
Total Wind Generation (kWh): 500723170.559
Total Genset Generation (kWh): 900680361.642
Total Battery Generation (kWh): -57465.129
Total CO2 emissions (tonne): 589495.297
System NPC ($): 121380055.708
System LCOE ($/kWh): 0.257
Deficit Hours Fixed: 7225.000
Total Load Reduction (kWh): 0.000
Total Load Served (kWh): 1969558247.650
Total Charging Prevented (kWh): 14047029.131
Demand Not Served (kWh): 0.000
Load Reduction Percentage: 0.000
Demand Met Percentage: 100.000
Project Lifetime (years): 25.000

Results saved to ../simulation_results_modularised/combined_results_without_DR/lowest_cost_configs_chunk_0_PDR.csv

Optimization complete. Summary of results:
Total locations processed: 3
Average LCOE: 0.